# GasNotFound
**Authors: Gaurav, Houston, Insiya, and Maya**
# **Vehicle Population Prediction Project**

## Project Goal

This project aims to predict the vehicle population in 2025 and future years by using data recorded from 2019 to 2024. The insights will help understand future fuel demand by predicting the number of miles driven and the types of vehicles on the road.

## Project Parts

The project is divided into the following parts:

**Part 1: Data Exploration and Cleaning**
* Explore the dataset and understand its structure.
* Clean the data by handling missing values and inconsistencies.
* Perform exploratory data analysis to uncover insights and patterns.

**Part 2: Feature Engineering**
* Create new features from existing ones to improve model performance.
* Select the most relevant features for prediction.

**Part 3: Model Selection and Training**
* Select an appropriate machine learning model for prediction.
* Train the model using the prepared data.

**Part 4: Model Evaluation and Prediction**
* Evaluate the model's performance using appropriate metrics.
* Use the trained model to predict the vehicle population in 2025.

**Part 5: Visualization and Reporting**
* Visualize the results using charts and graphs.
* Prepare a report summarizing the findings and insights.

## Dataset Description

The dataset contains information about vehicles registered between 2019 and 2024, including:
* Model Year
* Vehicle Type
* Fuel Type
* Number of Vehicles Registered to the Same Address


Let's begin with Part 1: Data Exploration and Cleaning.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving scoring.xlsx to scoring.xlsx
Saving training.xlsx to training.xlsx


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import pandas for data manipulation
import pandas as pd
import io
df = pd.read_excel(io.BytesIO(uploaded['training.xlsx'])) #read in
df_2 = pd.read_excel(io.BytesIO(uploaded['scoring.xlsx'])) #read in

In [ ]:
df_hc = pd.DataFrame(df)
df_scoring_sheet = pd.DataFrame(df_2)
print(df_hc.head(5))

   Date Vehicle Category      GVWR Class Fuel Type  Model Year  \
0  2019                P  Not Applicable  Gasoline      2020.0   
1  2020                P  Not Applicable  Gasoline      2020.0   
2  2021                P  Not Applicable  Gasoline      2020.0   
3  2019                P  Not Applicable  Gasoline      2019.0   
4  2019                P  Not Applicable  Gasoline      2018.0   

  Fuel Technology Electric Mile Range  \
0             ICE      Not Applicable   
1             ICE      Not Applicable   
2             ICE      Not Applicable   
3             ICE      Not Applicable   
4             ICE      Not Applicable   

  Number of Vehicles Registered at the Same Address     Region  \
0                                                ≥4  Statewide   
1                                                 1  Statewide   
2                                                 1  Statewide   
3                                                ≥4  Statewide   
4                         

In [ ]:
#This is the hard coding DF

#These are the mapping vectors
fuel_type_mapping = {
    "Diesel": [1, 0, 0, 0, 0, 0],
    "Gasoline": [0, 1, 0, 0, 0, 0],
    "Natural Gas": [0, 0, 1, 0, 0, 0],
    "Electric": [0, 0, 0, 1, 0, 0],
    "Hydrogen": [0, 0, 0, 0, 1, 0],
    "Unknown": [0, 0, 0, 0, 0, 1]
}

fuel_technology_mapping = {
    "BEV": [1, 0, 0, 0],
    "FCEV": [0, 1, 0, 0],
    "ICE": [0, 0, 1, 0],
    "PHEV": [0, 0, 0, 1]
}


df_hc[['Diesel', 'Gasoline', 'Natural Gas', 'Electric', 'Hydrogen', 'Unknown']] = df_hc['Fuel Type'].apply(lambda x: pd.Series(fuel_type_mapping[x]))
df_hc[['BEV', 'FCEV', 'ICE', 'PHEV']] = df_hc['Fuel Technology'].apply(lambda x: pd.Series(fuel_technology_mapping[x]))

df_scoring_sheet[['Diesel', 'Gasoline', 'Natural Gas', 'Electric', 'Hydrogen', 'Unknown']] = df_scoring_sheet['Fuel Type'].apply(lambda x: pd.Series(fuel_type_mapping[x]))
df_scoring_sheet[['BEV', 'FCEV', 'ICE', 'PHEV']] = df_scoring_sheet['Fuel Technology'].apply(lambda x: pd.Series(fuel_technology_mapping[x]))


# Display the result
df_hc.info()
"""
we need to solve question one (the population of cars for 2024)
I have converted it all to one hard coded
hence why vehicle population and below are not type int64 (as well as the date recorded / model year)
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41053 entries, 0 to 41052
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Date                                               41053 non-null  int64  
 1   Vehicle Category                                   41053 non-null  object 
 2   GVWR Class                                         41053 non-null  object 
 3   Fuel Type                                          41053 non-null  object 
 4   Model Year                                         40450 non-null  float64
 5   Fuel Technology                                    41053 non-null  object 
 6   Electric Mile Range                                41053 non-null  object 
 7   Number of Vehicles Registered at the Same Address  41053 non-null  object 
 8   Region                                             41053 non-null  object 
 9   Vehicl

'\nwe need to solve question one (the population of cars for 2024)\nI have converted it all to one hard coded\nhence why vehicle population and below are not type int64 (as well as the date recorded / model year)\n'

In [ ]:
df_hc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41053 entries, 0 to 41052
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Date                                               41053 non-null  int64  
 1   Vehicle Category                                   41053 non-null  object 
 2   GVWR Class                                         41053 non-null  object 
 3   Fuel Type                                          41053 non-null  object 
 4   Model Year                                         40450 non-null  float64
 5   Fuel Technology                                    41053 non-null  object 
 6   Electric Mile Range                                41053 non-null  object 
 7   Number of Vehicles Registered at the Same Address  41053 non-null  object 
 8   Region                                             41053 non-null  object 
 9   Vehicl

In [ ]:
# For df_hc
df_hc["Model Year"] = df_hc.groupby("Vehicle Category")["Model Year"].ffill().bfill()

df_scoring_sheet["Model Year"] = df_scoring_sheet.groupby("Vehicle Category")["Model Year"].ffill().bfill()

df_hc["Model Year"] = df_hc["Model Year"].astype(int)
df_scoring_sheet["Model Year"] = df_scoring_sheet["Model Year"].astype(int)

In [ ]:
df_hc['GVWR Class'] = df_hc['GVWR Class'].replace('Not Applicable', 0)
print(df_hc['GVWR Class'].unique())

df_scoring_sheet['GVWR Class'] = df_scoring_sheet['GVWR Class'].replace('Not Applicable', 0)
print(df_scoring_sheet['GVWR Class'].unique())

[0 1 2 'Unknown' 8 3 5 6 7 4]
[0 1 2 'Unknown' 8 5 3 6 7 4]


In [ ]:

# Convert 'GVWR Class' values based on 'Vehicle Category'
def convert_gvwr(row):
    if row['GVWR Class'] in ['Unknown', 'Not Applicable']:
        if row['Vehicle Category'] in ['T1', 'T2']:
            return 1
        elif row['Vehicle Category'] in ['T3', 'T4']:
            return 2
    return row['GVWR Class']

df_hc['GVWR Class'] = df_hc.apply(convert_gvwr, axis=1)

gvwr_by_category = df_hc.groupby('Vehicle Category')['GVWR Class'].value_counts().unstack(fill_value=0)
gvwr_by_category


GVWR Class,0,1,2,3,4,5,6,7,8,Unknown
Vehicle Category,,,,,,,,,,
B,2306,0,0,0,0,0,0,0,0,0
BS,2175,0,0,0,0,0,0,0,0,0
BT,1922,0,0,0,0,0,0,0,0,0
MC,2038,0,0,0,0,0,0,0,0,0
MH,1961,0,0,0,0,0,0,0,0,0
P,3800,0,0,0,0,0,0,0,0,0
T1,0,3012,0,0,0,0,0,0,0,0
T2,0,3499,0,0,0,0,0,0,0,0
T3,0,0,4078,0,0,0,0,0,0,0


In [ ]:
def convert_gvwr(row):
    if row['GVWR Class'] in ['Unknown', 'Not Applicable']:
        if row['Vehicle Category'] in ['T1', 'T2']:
            return 1
        elif row['Vehicle Category'] in ['T3', 'T4']:
            return 2
    return row['GVWR Class']

df_scoring_sheet['GVWR Class'] = df_scoring_sheet.apply(convert_gvwr, axis=1)

gvwr_by_category_scoring = df_scoring_sheet.groupby('Vehicle Category')['GVWR Class'].value_counts().unstack(fill_value=0)
print(gvwr_by_category_scoring)

GVWR Class          0    1    2    3    4    5    6    7    8  Unknown
Vehicle Category                                                      
B                 460    0    0    0    0    0    0    0    0        0
BS                433    0    0    0    0    0    0    0    0        0
BT                370    0    0    0    0    0    0    0    0        0
MC                388    0    0    0    0    0    0    0    0        0
MH                392    0    0    0    0    0    0    0    0        0
P                 775    0    0    0    0    0    0    0    0        0
T1                  0  295    0    0    0    0    0    0    0        0
T2                  0  473    0    0    0    0    0    0    0        0
T3                  0    0  708    0    0    0    0    0    0        0
T4                  0    0  705    0    0    0    0    0    0        0
T5                  0    0    0  347    0    0    0    0    0        0
T6                  0    0    0    0  333  315  388  371    0      411
T7    

In [ ]:
import pandas as pd
import numpy as np

t6_unknown_gvwr = df_hc[(df_hc['Vehicle Category'] == 'T6') & (df_hc['GVWR Class'] == 'Unknown')]

indices_to_modify = t6_unknown_gvwr.index


# Assign 407 rows to each of the GVWR classes 4, 5, 6, and 7
for i, gvwr_class in enumerate([4, 5, 6, 7]):
    start_index = i * 407
    end_index = start_index + 407
    df_hc.loc[indices_to_modify[start_index:end_index], 'GVWR Class'] = gvwr_class

gvwr_by_category = df_hc.groupby('Vehicle Category')['GVWR Class'].value_counts().unstack(fill_value=0)
print(gvwr_by_category)

GVWR Class           0     1     2     3     4     5     6     7     8
Vehicle Category                                                      
B                 2306     0     0     0     0     0     0     0     0
BS                2175     0     0     0     0     0     0     0     0
BT                1922     0     0     0     0     0     0     0     0
MC                2038     0     0     0     0     0     0     0     0
MH                1961     0     0     0     0     0     0     0     0
P                 3800     0     0     0     0     0     0     0     0
T1                   0  3012     0     0     0     0     0     0     0
T2                   0  3499     0     0     0     0     0     0     0
T3                   0     0  4078     0     0     0     0     0     0
T4                   0     0  3925     0     0     0     0     0     0
T5                   0     0     0  1778     0     0     0     0     0
T6                   0     0     0     0  2000  2022  2368  2315     0
T7    

In [ ]:
import pandas as pd
import numpy as np

t6_unknown_gvwr = df_scoring_sheet[(df_scoring_sheet['Vehicle Category'] == 'T6') & (df_scoring_sheet['GVWR Class'] == 'Unknown')]

indices_to_modify = t6_unknown_gvwr.index

for i, gvwr_class in enumerate([4, 5, 6, 7]):
    start_index = i * 407
    end_index = start_index + 407
    df_scoring_sheet.loc[indices_to_modify[start_index:end_index], 'GVWR Class'] = gvwr_class

gvwr_by_category_scoring = df_scoring_sheet.groupby('Vehicle Category')['GVWR Class'].value_counts().unstack(fill_value=0)
print(gvwr_by_category_scoring)

GVWR Class          0    1    2    3    4    5    6    7    8
Vehicle Category                                             
B                 460    0    0    0    0    0    0    0    0
BS                433    0    0    0    0    0    0    0    0
BT                370    0    0    0    0    0    0    0    0
MC                388    0    0    0    0    0    0    0    0
MH                392    0    0    0    0    0    0    0    0
P                 775    0    0    0    0    0    0    0    0
T1                  0  295    0    0    0    0    0    0    0
T2                  0  473    0    0    0    0    0    0    0
T3                  0    0  708    0    0    0    0    0    0
T4                  0    0  705    0    0    0    0    0    0
T5                  0    0    0  347    0    0    0    0    0
T6                  0    0    0    0  740  319  388  371    0
T7                  0    0    0    0    0    0    0    0  382


In [ ]:
# display(df_hc)
df_hc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41053 entries, 0 to 41052
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Date                                               41053 non-null  int64 
 1   Vehicle Category                                   41053 non-null  object
 2   GVWR Class                                         41053 non-null  object
 3   Fuel Type                                          41053 non-null  object
 4   Model Year                                         41053 non-null  int64 
 5   Fuel Technology                                    41053 non-null  object
 6   Electric Mile Range                                41053 non-null  object
 7   Number of Vehicles Registered at the Same Address  41053 non-null  object
 8   Region                                             41053 non-null  object
 9   Vehicle Populatio

In [ ]:
df_hc['light_weight'] = df_hc['GVWR Class'].apply(lambda x: 1 if 0 <= x <= 2 else 0)
df_hc['medium_weight'] = df_hc['GVWR Class'].apply(lambda x: 1 if 3 <= x <= 5 else 0)
df_hc['heavy_weight'] = df_hc['GVWR Class'].apply(lambda x: 1 if 6 <= x <= 8 else 0)

In [ ]:
df_scoring_sheet['light_weight'] = df_scoring_sheet['GVWR Class'].apply(lambda x: 1 if 0 <= x <= 2 else 0)
df_scoring_sheet['medium_weight'] = df_scoring_sheet['GVWR Class'].apply(lambda x: 1 if 3 <= x <= 5 else 0)
df_scoring_sheet['heavy_weight'] = df_scoring_sheet['GVWR Class'].apply(lambda x: 1 if 6 <= x <= 8 else 0)

In [ ]:
df_hc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41053 entries, 0 to 41052
Data columns (total 23 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Date                                               41053 non-null  int64 
 1   Vehicle Category                                   41053 non-null  object
 2   GVWR Class                                         41053 non-null  object
 3   Fuel Type                                          41053 non-null  object
 4   Model Year                                         41053 non-null  int64 
 5   Fuel Technology                                    41053 non-null  object
 6   Electric Mile Range                                41053 non-null  object
 7   Number of Vehicles Registered at the Same Address  41053 non-null  object
 8   Region                                             41053 non-null  object
 9   Vehicle Populatio

In [ ]:

mpg_values = {
    'ICE': 34,
    'BEV': 95,
    'FCEV': 50,
    'PHEV': 42
}

df_hc['mpg'] = df_hc['Fuel Technology'].map(mpg_values)

df_hc.head()


import numpy as np

mileage_distribution = {
    'ICE': ([5000, 7500, 12500, 17500, 22500, 25000], [0.28, 0.30, 0.26, 0.07, 0.04, 0.05]),
    'FCEV': ([5000, 7500, 12500, 17500, 22500, 25000], [0.03, 0.29, 0.44, 0.15, 0.06, 0.03]),
    'BEV': 9000,
    'PHEV': 11000
}


def assign_mileage(fuel_type):
    if fuel_type in ['BEV', 'PHEV']:  # fixed mileage for BEV & PHEV
        return mileage_distribution[fuel_type]
    elif fuel_type in ['ICE', 'FCEV']:  # Random sampling
        miles, probs = mileage_distribution[fuel_type]
        return np.random.choice(miles, p=probs)
    else:
        return np.nan

df_hc['Average Miles Traveled Annually'] = df_hc['Fuel Technology'].apply(assign_mileage)


# first few rows to verify
df_hc.head()


,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population,...,Unknown,BEV,FCEV,ICE,PHEV,light_weight,medium_weight,heavy_weight,mpg,Average Miles Traveled Annually
0,2019,P,0,Gasoline,2020,ICE,Not Applicable,≥4,Statewide,395883,...,0,0,0,1,0,1,0,0,34,12500
1,2020,P,0,Gasoline,2020,ICE,Not Applicable,1,Statewide,370954,...,0,0,0,1,0,1,0,0,34,12500
2,2021,P,0,Gasoline,2020,ICE,Not Applicable,1,Statewide,349406,...,0,0,0,1,0,1,0,0,34,12500
3,2019,P,0,Gasoline,2019,ICE,Not Applicable,≥4,Statewide,348475,...,0,0,0,1,0,1,0,0,34,5000
4,2019,P,0,Gasoline,2018,ICE,Not Applicable,≥4,Statewide,333296,...,0,0,0,1,0,1,0,0,34,22500


In [ ]:

#  the MPG values for each fuel technology
mpg_values = {
    'ICE': 34,
    'BEV': 95,
    'FCEV': 50,
    'PHEV': 42
}

df_scoring_sheet['mpg'] = df_scoring_sheet['Fuel Technology'].map(mpg_values)

df_scoring_sheet.head()


import numpy as np


mileage_distribution = {
    'ICE': ([5000, 7500, 12500, 17500, 22500, 25000], [0.28, 0.30, 0.26, 0.07, 0.04, 0.05]),
    'FCEV': ([5000, 7500, 12500, 17500, 22500, 25000], [0.03, 0.29, 0.44, 0.15, 0.06, 0.03]),
    'BEV': 9000,
    'PHEV': 11000
}

def assign_mileage(fuel_type):
    if fuel_type in ['BEV', 'PHEV']:
        return mileage_distribution[fuel_type]
    elif fuel_type in ['ICE', 'FCEV']:
        miles, probs = mileage_distribution[fuel_type]
        return np.random.choice(miles, p=probs)
    else:
        return np.nan

df_scoring_sheet['Average Miles Traveled Annually'] = df_scoring_sheet['Fuel Technology'].apply(assign_mileage)

df_scoring_sheet.head()


,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population,...,Unknown,BEV,FCEV,ICE,PHEV,light_weight,medium_weight,heavy_weight,mpg,Average Miles Traveled Annually
0,2024,P,0,Gasoline,2020,ICE,Not Applicable,≥4,Statewide,316065,...,0,0,0,1,0,1,0,0,34,12500
1,2024,P,0,Gasoline,2021,ICE,Not Applicable,≥4,Statewide,315986,...,0,0,0,1,0,1,0,0,34,7500
2,2024,P,0,Gasoline,2022,ICE,Not Applicable,≥4,Statewide,306487,...,0,0,0,1,0,1,0,0,34,12500
3,2024,P,0,Gasoline,2024,ICE,Not Applicable,≥4,Statewide,284754,...,0,0,0,1,0,1,0,0,34,7500
4,2024,P,0,Gasoline,2023,ICE,Not Applicable,≥4,Statewide,284153,...,0,0,0,1,0,1,0,0,34,17500


In [ ]:
df_hc.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41053 entries, 0 to 41052
Data columns (total 25 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Date                                               41053 non-null  int64 
 1   Vehicle Category                                   41053 non-null  object
 2   GVWR Class                                         41053 non-null  object
 3   Fuel Type                                          41053 non-null  object
 4   Model Year                                         41053 non-null  int64 
 5   Fuel Technology                                    41053 non-null  object
 6   Electric Mile Range                                41053 non-null  object
 7   Number of Vehicles Registered at the Same Address  41053 non-null  object
 8   Region                                             41053 non-null  object
 9   Vehicle Populatio

In [ ]:
df_hc['Total'] = df_hc.groupby('Model Year')['Vehicle Population'].transform('sum')

df_hc['Cumulative_Total'] = df_hc.sort_values('Model Year')['Total'].cumsum()

num_bins = 4
total_vehicles = df_hc['Cumulative_Total'].max()
bin_edges = np.linspace(0, total_vehicles, num_bins + 1)

df_hc['Model_Year_Range'] = pd.cut(df_hc['Cumulative_Total'], bins=bin_edges, labels=False) + 1


In [ ]:
df_scoring_sheet["Model Year"] = df_scoring_sheet.groupby("Vehicle Category")["Model Year"].ffill().bfill()

In [ ]:
df_scoring_sheet['Total'] = df_scoring_sheet.groupby('Model Year')['Vehicle Population'].transform('sum')

df_scoring_sheet['Cumulative_Total'] = df_scoring_sheet.sort_values('Model Year')['Total'].cumsum()

num_bins = 4
total_vehicles = df_scoring_sheet['Cumulative_Total'].max()
bin_edges = np.linspace(0, total_vehicles, num_bins + 1)

df_scoring_sheet['Model_Year_Range'] = pd.cut(df_hc['Cumulative_Total'], bins=bin_edges, labels=False) + 1


In [ ]:
df_hc_selected = df_hc[['Diesel', 'Gasoline', 'Natural Gas', 'Electric', 'Hydrogen', 'Unknown',
                        'BEV', 'FCEV', 'ICE', 'PHEV',
                        'Date', 'Vehicle Population', 'Model Year','light_weight','medium_weight','heavy_weight','mpg' ,'Average Miles Traveled Annually']]

df_scoring_sheet_selected = df_scoring_sheet[['Diesel', 'Gasoline', 'Natural Gas', 'Electric', 'Hydrogen', 'Unknown',
                        'BEV', 'FCEV', 'ICE', 'PHEV',
                        'Date', 'Vehicle Population', 'Model Year','light_weight','medium_weight','heavy_weight','mpg','Average Miles Traveled Annually']]

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, PoissonRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import xgboost as xgb
import catboost as cb

def prepare_features(df):
    df = df.copy()



    return df

numeric_features = [
    'Model Year', 'mpg', 'Average Miles Traveled Annually',
    'Diesel', 'Gasoline', 'Natural Gas', 'Electric', 'Hydrogen', 'Unknown',
    'BEV', 'FCEV', 'ICE', 'PHEV', 'light_weight', 'medium_weight', 'heavy_weight'
]

categorical_features = [
    'Vehicle Category', 'GVWR Class', 'Fuel Type', 'Fuel Technology', 'Region'
]

# Create preprocessing pipeline
def create_preprocessor():
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop'
    )

    return preprocessor

def create_model_pipeline(model_name):
    """Create a pipeline with preprocessing and model"""
    if model_name == "Random Forest":
        model = RandomForestRegressor(
            n_estimators=200,
            max_depth=15,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42
        )
    elif model_name == "Gradient Boosting":
        model = GradientBoostingRegressor(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=8,
            random_state=42
        )
    elif model_name == "SVR":
        model = SVR(
            kernel='rbf',
            C=1.0,
            epsilon=0.1
        )
    elif model_name == "Poisson":
        model = PoissonRegressor(
            alpha=0.5,
            max_iter=300,
            tol=1e-3
        )
    else:  # Linear Regression
        model = LinearRegression()

    pipeline = Pipeline(steps=[
        ('preprocessor', create_preprocessor()),
        ('regressor', model)
    ])

    return pipeline

print("Preparing training data...")
df_hc = prepare_features(df_hc)
print("Preparing scoring data...")
df_scoring_sheet = prepare_features(df_scoring_sheet)

feature_cols = [col for col in df_hc.columns if col != 'Vehicle Population']
X_train = df_hc[feature_cols]
y_train = df_hc['Vehicle Population']
X_scoring = df_scoring_sheet[feature_cols]
y_scoring = df_scoring_sheet['Vehicle Population']

preprocessor = create_preprocessor()
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_scoring_preprocessed = preprocessor.transform(X_scoring)

models = {
    "XGBoost": xgb.XGBRegressor(
        n_estimators=200,
        max_depth=7,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        min_child_weight=3,
        random_state=42
    ),
    "CatBoost": cb.CatBoostRegressor(
        iterations=200,
        learning_rate=0.1,
        depth=7,
        l2_leaf_reg=3,
        random_seed=42,
        verbose=False
    ),
    "Random Forest": create_model_pipeline("Random Forest"),
    "Gradient Boosting": create_model_pipeline("Gradient Boosting"),
    "SVR": create_model_pipeline("SVR"),
    "Poisson": create_model_pipeline("Poisson"),
    "Linear Regression": create_model_pipeline("Linear")
}

results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")

    if name in ["XGBoost", "CatBoost"]:
        model.fit(X_train_preprocessed, y_train)
        y_pred = model.predict(X_scoring_preprocessed)
    else:

        model.fit(X_train, y_train)
        y_pred = model.predict(X_scoring)

    rmse = np.sqrt(mean_squared_error(y_scoring, y_pred))
    results[name] = rmse
    print(f"{name} Scoring RMSE: {rmse:.4f}")

print("\nFinal Results (sorted by performance):")
sorted_results = dict(sorted(results.items(), key=lambda x: x[1]))
for name, rmse in sorted_results.items():
    print(f"{name:20} RMSE: {rmse:.4f}")

def print_feature_importance(name, model, preprocessor):
    if name in ["XGBoost", "CatBoost"]:
        feature_names = (
            numeric_features +
            preprocessor.named_transformers_['cat']
            .named_steps['onehot']
            .get_feature_names_out(categorical_features).tolist()
        )
        try:
            importances = model.feature_importances_
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': importances
            }).sort_values('importance', ascending=False)
            print(f"\nTop 10 most important features for {name}:")
            print(importance_df.head(10))
        except:
            print(f"\nCouldn't get feature importance for {name}")

for name, model in models.items():
    if name in ["XGBoost", "CatBoost"]:
        print_feature_importance(name, model, preprocessor)

Preparing training data...
Preparing scoring data...

Training XGBoost...
XGBoost Scoring RMSE: 10791.4073

Training CatBoost...
CatBoost Scoring RMSE: 10665.2287

Training Random Forest...
Random Forest Scoring RMSE: 10756.8602

Training Gradient Boosting...
Gradient Boosting Scoring RMSE: 10850.2416

Training SVR...
SVR Scoring RMSE: 19745.0764

Training Poisson...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_glm/glm.py:285: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


Poisson Scoring RMSE: 12618.6466

Training Linear Regression...
Linear Regression Scoring RMSE: 17632.6983

Final Results (sorted by performance):
CatBoost             RMSE: 10665.2287
Random Forest        RMSE: 10756.8602
XGBoost              RMSE: 10791.4073
Gradient Boosting    RMSE: 10850.2416
Poisson              RMSE: 12618.6466
Linear Regression    RMSE: 17632.6983
SVR                  RMSE: 19745.0764

Top 10 most important features for XGBoost:
                  feature  importance
1                     mpg    0.190034
42    Fuel Technology_ICE    0.166701
20     Vehicle Category_P    0.138528
4                Gasoline    0.119544
11                    ICE    0.080974
37     Fuel Type_Gasoline    0.061709
0              Model Year    0.040872
16    Vehicle Category_BS    0.036961
13           light_weight    0.028656
39  Fuel Type_Natural Gas    0.026360

Top 10 most important features for CatBoost:
                            feature  importance
20               Vehicle Categ

In [ ]:
# Dictionary to store predictions for each model
predictions = {}

for name, model in models.items():
    print(f"\nTraining {name}...")

    if name in ["XGBoost", "CatBoost"]:
        model.fit(X_train_preprocessed, y_train)
        y_pred = model.predict(X_scoring_preprocessed)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_scoring)

    predictions[name] = y_pred

    rmse = np.sqrt(mean_squared_error(y_scoring, y_pred))
    results[name] = rmse
    print(f"{name} Scoring RMSE: {rmse:.4f}")

for name, y_pred in predictions.items():
    df_scoring_sheet[f'Predicted_{name}'] = y_pred

print("\nTest data with predictions:")
print(df_scoring_sheet.head())

print("\nFinal Results (sorted by performance):")
sorted_results = dict(sorted(results.items(), key=lambda x: x[1]))
for name, rmse in sorted_results.items():
    print(f"{name:20} RMSE: {rmse:.4f}")

for name, model in models.items():
    if name in ["XGBoost", "CatBoost"]:
        print_feature_importance(name, model, preprocessor)


Training XGBoost...
XGBoost Scoring RMSE: 10791.4073

Training CatBoost...
CatBoost Scoring RMSE: 10665.2287

Training Random Forest...
Random Forest Scoring RMSE: 10756.8602

Training Gradient Boosting...
Gradient Boosting Scoring RMSE: 10850.2416

Training SVR...
SVR Scoring RMSE: 19745.0764

Training Poisson...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_glm/glm.py:285: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


Poisson Scoring RMSE: 12618.6466

Training Linear Regression...
Linear Regression Scoring RMSE: 17632.6983

Test data with predictions:
   Date Vehicle Category GVWR Class Fuel Type  Model Year Fuel Technology  \
0  2024                P          0  Gasoline        2020             ICE   
1  2024                P          0  Gasoline        2021             ICE   
2  2024                P          0  Gasoline        2022             ICE   
3  2024                P          0  Gasoline        2024             ICE   
4  2024                P          0  Gasoline        2023             ICE   

  Electric Mile Range Number of Vehicles Registered at the Same Address  \
0      Not Applicable                                                ≥4   
1      Not Applicable                                                ≥4   
2      Not Applicable                                                ≥4   
3      Not Applicable                                                ≥4   
4      Not Applicable     

In [ ]:
# Add predictions as new columns to df_scoring_sheet
for name, y_pred in predictions.items():
    df_scoring_sheet[f'Predicted_{name}'] = y_pred

fuel_tech_predictions = df_scoring_sheet.groupby('Fuel Type')[['Predicted_XGBoost', 'Predicted_CatBoost', 'Predicted_Random Forest',
                                                                     'Predicted_Gradient Boosting', 'Predicted_SVR', 'Predicted_Poisson',
                                                                     'Predicted_Linear Regression']].sum()

print("Sum of predicted values by Fuel Technology:")

pd.set_option('display.float_format', '{:.0f}'.format)

print(fuel_tech_predictions)

Sum of predicted values by Fuel Technology:
             Predicted_XGBoost  Predicted_CatBoost  Predicted_Random Forest  \
Fuel Type                                                                     
Diesel                 1072156             1045066                  1108824   
Electric                241915              245788                   329206   
Gasoline              25651916            25490124                 26035166   
Hydrogen                  2007                5820                     6657   
Natural Gas              24430               77945                    33565   

             Predicted_Gradient Boosting  Predicted_SVR  Predicted_Poisson  \
Fuel Type                                                                    
Diesel                           1101753         217685            1190163   
Electric                          296015          10384             244033   
Gasoline                        26068413         897664           30335955   
Hydrogen    